In [3]:
d = {
  "Copied": "已复制",
  "Copy": "复制",
  "Copy to clipboard": "复制到剪贴板",
  "Copy to clipboard: Ctrl+C, Enter": "复制到剪贴板：Ctrl+C, Enter",
  "Copy Failed": "复制失败",
  "Search Icon": "搜索图标",
  "All Icons Sets": "全部图标库",
  "Select All": "全选",
  "OK": "确定",
  "No icons found": "没有找到图标",
  "Color": "颜色",
  "Size": "大小",
  "Download": "下载",
  "Background": "背景",
  "Icon": "图标",
  "Radius": "半径",
  "Padding": "内边距",
  "Border": "边框",
  "Shadow": "阴影",
  "Opacity": "透明度",
  "Style": "样式",
  "Margin": "外边距",
  "CopySVG": "复制 SVG",
  "CopyReactComponent": "复制 React 组件",
  "CopyReactImport": "复制 React 导入",
  "CopyName": "复制名称",
  "CopyDataURI": "复制 Data URI",
  "CopyPNG": "复制 PNG",
  "DownloadPNG": "下载 PNG",
  "DownloadSVG": "下载 SVG",
  "DownloadReact": "下载 React 组件",
  "DownloadVue": "下载 Vue 组件",
  "DownloadSvelte": "下载 Svelte 组件",
  "DownloadICO": "下载 ICO"
}

for k, v in d.items():
    d[k] = k

import json

print(json.dumps(d, ensure_ascii=False, indent=2))

{
  "Copied": " copied",
  "Copy": " copy",
  "Copy to clipboard": " copy to clipboard",
  "Copy to clipboard: Ctrl+C, Enter": " copy to clipboard:  ctrl+ c,  enter",
  "Copy Failed": " copy  failed",
  "Search Icon": " search  icon",
  "All Icons Sets": " all  icons  sets",
  "Select All": " select  all",
  "OK": " o k",
  "No icons found": " no icons found",
  "Color": " color",
  "Size": " size",
  "Download": " download",
  "Background": " background",
  "Icon": " icon",
  "Radius": " radius",
  "Padding": " padding",
  "Border": " border",
  "Shadow": " shadow",
  "Opacity": " opacity",
  "Style": " style",
  "Margin": " margin",
  "CopySVG": " copy s v g",
  "CopyReactComponent": " copy react component",
  "CopyReactImport": " copy react import",
  "CopyName": " copy name",
  "CopyDataURI": " copy data u r i",
  "CopyPNG": " copy p n g",
  "DownloadPNG": " download p n g",
  "DownloadSVG": " download s v g",
  "DownloadReact": " download react",
  "DownloadVue": " download vue",
